In [1]:
from transformer import Transformer
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
import numpy as np

%load_ext autoreload
%autoreload 2


In [2]:
# https://www.statmt.org/europarl/
english_file = 'da-en/europarl-v7.da-en.en' # replace this path with appropriate one
danish_file = 'da-en/europarl-v7.da-en.da' # replace this path with appropriate one

START_TOKEN = '<START>'
PADDING_TOKEN = '<PAD>'
END_TOKEN = '<END>'

danish_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ';',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', '@',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'æ', 'ø', 'å', 'é',
                        'y', 'z', '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ';',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', "]", '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [3]:
index_to_danish = {k:v for k,v in enumerate(danish_vocabulary)}
danish_to_index = {v:k for k,v in enumerate(danish_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [4]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(danish_file, 'r') as file:
    dk_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
dk_sentences = dk_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
dk_sentences = [sentence.rstrip('\n').lower() for sentence in dk_sentences]

In [5]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):

    for token in list(set(sentence)):
        if token not in vocab:
            print(token)
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(dk_sentences)):
    danish_sentence, english_sentence = dk_sentences[index], english_sentences[index]
    if is_valid_length(danish_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(danish_sentence, danish_vocabulary) \
      and is_valid_tokens(english_sentence, english_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(dk_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

í
ó
ó
ä
ó
ä
ä
º
º
º
é
ä
­
á
ü
è
ï
ö
ø
ø
ô
è
ä
ê
ü
ü
ê
­
ö
­
ø
é
§
é
­
ó
ó
ö
ö
ö
ü
ü
é
é
é
é
é
ò
ò
ö
ö
é
é
ö
ö
ö
ö
ö
ü
ü
ü
ü
ö
ö
ü
é
ê
ö
ö
ö
ö
ö
ö
ï
à
é
à
ü
£
ç
ç
ç
ü
ö
ä
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ó
ö
ö
è
è
á
é
è
è
¾
ü
ö
è
è
è
è
è
è
ö
ö
ö
ö
ö
á
ö
ö
ö
ä
ö
ö
ö
è
ö
ö
ö
ö
ö
ö
è
è
é
é
é
é
à
è
è
è
è
è
è
ö
á
ö
ö
ö
è
è
è
è
è
à
á
á
á
è
ö
á
á
º
ü
é
ü
ö
ö
á
á
é
é
é
é
é
ô
à
ö
ü
ö
ö
ö
ö
ö
í
é
ö
é
û
é
é
é
í
í
í
á
ö
ó
ö
è
è
é
é
é
ê
ö
ö
é
ñ
ñ
ö
ö
è
ö
í
ö
ö
ü
ü
ü
ü
é
é
í
ó
é
ü
ê
á
ï
é
ü
ô
î
º
è
é
é
è
º
ö
ö
ä
ó
ó
ó
à
ö
ó
í
ö
ö
í
í
í
í
é
é
é
ê
á
á
í
ó
í
ö
ö
ö
ö
ü
ö
è
ü
ü
ü
ê
µ
µ
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
ö
è
è
è
è
­
è
è
è
è
è
í
è
è
è
è
è
è
ö
ö
ö
ö
è
ï
ö
è
ö
ö
à
ö
ö
á
·
·
·
è
á
á
á
ö
ö
ö
ö
è
ö
í
í
ö
ä
é
é
é
à
à
ê
ö
à
ö
ö
ö
á
í
á
ö
á
á
ó
ö
ê
ö
à
à
ä
í
å
ï
ï
ó
ó
à
è
á
ó
ó
è
è
ö
ö
ü
á
ö
ä
ä
ä
ä
è
à
ß
ß
ß
ü
ü
ä
é
ü
é
ö
ö
é
é
é
ö
è
é
ó
ö
é
é
é
é
é
é
é
é
ö
ó
£
ö
é
ö
ö
ö
á
é
é
é
ö
ä
ö
í
í
í
í
í
ç
ç
ö
ö
ö
ö
ö
í
í
é
í
í
ó
ó
ó
é
ó
ä
ö
à
ö
­
è
è
­
­
­
­
ö
­
ö
á
­
­
ö
ö
­
­
­
­
­
­
­
­
ö
­
ö
ö
è
é
á
ü
ü
ö
ö
ç
ö
ç


In [6]:
dk_sentences = [dk_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [7]:
d_model = 512
num_heads = 8
ffn_hidden = 2048
num_layers = 1
batch_size = 30
drop_prob = 0.1
max_sequence_length = 200
dk_vocab_size = len(danish_vocabulary)

transformer = Transformer(d_model, 
                 ffn_hidden,       
                 num_heads,
                 drop_prob, 
                 num_layers,
                 max_sequence_length,
                 dk_vocab_size,
                 english_to_index,
                 danish_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN)

In [8]:
class TextDataset(Dataset):

    def __init__(self, english_sentences, dk_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = dk_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]
    


dataset = TextDataset(english_sentences, dk_sentences)
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [9]:
criterion = nn.CrossEntropyLoss(ignore_index=danish_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
NEG_INFTY = -1e9

def create_masks(eng_batch, dk_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, dk_sentence_length = len(eng_batch[idx]), len(dk_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      dk_chars_to_padding_mask = np.arange(dk_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, dk_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, dk_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, dk_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [11]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

dk_vocab_size = len(danish_vocabulary)

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, dk_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, dk_batch)
        optim.zero_grad()
        dk_predictions = transformer(eng_batch,
                                     dk_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(dk_batch, start_token=False, end_token=True)
        loss = criterion(
            dk_predictions.view(-1, dk_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == danish_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Danish Translation: {dk_batch[0]}")
            dk_sentence_predicted = torch.argmax(dk_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in dk_sentence_predicted:
              if idx == danish_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_danish[idx.item()]
            print(f"Danish Prediction: {predicted_sentence}")

            transformer.eval()
            dk_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, dk_sentence)
                predictions = transformer(eng_sentence,
                                          dk_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_danish[next_token_index]
                dk_sentence = (dk_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {dk_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 4.477841377258301
English: resumption of the session
Danish Translation: genoptagelse af sessionen
Danish Prediction: u   eo    !   ,uu!o   fffofoovoooffo!!éé!! 5!é-o  oé   a!åååå1  o@ 19!!!1!nn9!9o;;1!!!7oo?åyaa7   !aaa!af@af     0   f |;;;; aofn na>aaaaaan  a@nnnn99% p0 "gpnpfpp!gp!9!!b9!!!99n!f9!f!9!9|9n9ff9nå|uåå
Evaluation translation (should we go to the mall?) : ('                               n n   ee                          e      e           ee              aaa                              aaaaae      nnn n             gg         a 9    9 99n n nnn      ee',)
-------------------------------------------


KeyboardInterrupt: 